# SQL Skills Practice: Exploring the Chinook Database

## Introduction
In this guided project, I will be honing my SQL skills by exploring the Chinook database. The Chinook database, provided as a SQLite file called `chinook.db`, offers valuable insights into a digital media store's operations.

## Database Overview
The Chinook database schema encompasses tables representing various aspects of the digital media store, including customers, invoices, tracks, albums, and genres.

## Project Objectives
My goal is to leverage SQL queries to address specific business questions and extract actionable insights from the Chinook database. By analyzing sales data, track popularity, customer behavior, and other metrics, I aim to uncover patterns and make informed recommendations for business optimization.

## Query Writing Strategies
Throughout the project, I will adopt several strategies to write effective SQL queries:
- **Iterative Approach**: I will write queries in stages, testing and validating each step to ensure accuracy.
- **Debugging Techniques**: If faced with unexpected results, I will break down queries into smaller components for debugging.
- **Data Validation**: I will use separate queries to validate underlying data and calculations, ensuring the integrity of my analyses.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

# Query the database to get a list of all tables in db

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [4]:
%%sql
SELECT *
FROM genre
Limit 10;

Done.


genre_id,name
1,Rock
2,Jazz
3,Metal
4,Alternative & Punk
5,Rock And Roll
6,Blues
7,Latin
8,Reggae
9,Pop
10,Soundtrack


## Which genres sell best in the USA?
### Write a query that returns each genre, with the number of tracks sold in the USA:
- in absolute numbers
- in percentages.

In [5]:
%%sql
SELECT * 
FROM invoice
where billing_country = 'USA'
Limit 10;

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83
9,18,2017-01-18 00:00:00,627 Broadway,New York,NY,USA,10012-2612,8.91
14,25,2017-01-23 00:00:00,319 N. Frances Street,Madison,WI,USA,53703,0.99
17,20,2017-01-28 00:00:00,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,10.89
18,24,2017-02-02 00:00:00,162 E Superior Street,Chicago,IL,USA,60611,4.95
42,19,2017-03-18 00:00:00,1 Infinite Loop,Cupertino,CA,USA,95014,8.91
43,22,2017-03-19 00:00:00,120 S Orange Ave,Orlando,FL,USA,32801,6.93
44,21,2017-03-20 00:00:00,801 W 4th Street,Reno,NV,USA,89503,11.879999999999999


In [12]:
%%sql
SELECT *
from invoice_line
limit 10

Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1
6,1,1163,0.99,1
7,1,1164,0.99,1
8,1,1165,0.99,1
9,1,1166,0.99,1
10,1,1167,0.99,1


In [14]:
%%sql
SELECT invoice_id, count(*)
FROM invoice_line
group by 1
order by count(*) desc
Limit 50;

Done.


invoice_id,count(*)
183,24
31,20
92,20
362,20
526,20
276,19
54,18
99,18
141,18
175,18


In [15]:
%%sql
WITH usa AS (
    SELECT 
        g.name as genre,
        ROUND(SUM(il.unit_price), 2) AS total,
        COUNT(distinct invoice_line_id) AS no_sold
    FROM 
        invoice AS i
        JOIN invoice_line AS il USING (invoice_id)
        JOIN playlist_track AS p ON il.track_id = p.track_id
        JOIN track AS t ON p.track_id = t.track_id
        JOIN genre AS g ON t.genre_id = g.genre_id
    WHERE 
        billing_country = 'USA'
    GROUP BY 
        g.name
    ORDER BY 
        total DESC
)

SELECT *
FROM usa;


Done.


genre,total,no_sold
Rock,1371.15,561
Alternative & Punk,287.1,130
Metal,287.1,124
R&B/Soul,115.83,53
Blues,101.97,36
Alternative,71.28,35
Hip Hop/Rap,59.4,20
Latin,53.46,22
Pop,43.56,22
Jazz,28.71,14


In [20]:
%%sql
WITH usa AS (
    SELECT 
       g.name as genre, 
       COUNT(DISTINCT invoice_line_id) as no_tracks_sold
    FROM 
        invoice AS i
        JOIN invoice_line AS il
        ON i.invoice_id = il.invoice_id
        JOIN customer c on i.customer_id = c.customer_id
        JOIN playlist_track AS p 
        ON il.track_id = p.track_id
        JOIN track AS t ON p.track_id = t.track_id
        JOIN genre AS g ON t.genre_id = g.genre_id
    WHERE 
        c.country = 'USA'
    GROUP BY 
        g.name
    ORDER BY 
        no_tracks_sold DESC
)

SELECT *,
    round(no_tracks_sold * 100.0/(SELECT SUM(no_tracks_sold) FROM usa),1) as perct
FROM usa;


Done.


genre,no_tracks_sold,perct
Rock,561,53.4
Alternative & Punk,130,12.4
Metal,124,11.8
R&B/Soul,53,5.0
Blues,36,3.4
Alternative,35,3.3
Latin,22,2.1
Pop,22,2.1
Hip Hop/Rap,20,1.9
Jazz,14,1.3


### Extract the columns I need to find the employee with the highest sells within the company and any other relevant information

In [10]:
%%sql
with top_employees as
(SELECT e.employee_id, 
        e.first_name ||' ' || e.last_name as employee,
        e.title,
        e.reports_to,
        SUBSTR(e.hire_date,1,10) AS hire_date, e.hire_date string,
        e.country, i.total
FROM employee e
JOIN customer c
ON e.employee_id = c.support_rep_id
JOIN invoice i
ON c.customer_id = i.customer_id)

SELECT employee_id, employee, title, reports_to, hire_date, country,
    round(sum(total),2) as total_sales
FROM top_employees
GROUP BY 1,2,3,4,5,6;



Done.


employee_id,employee,title,reports_to,hire_date,country,total_sales
3,Jane Peacock,Sales Support Agent,2,2017-04-01,Canada,1731.51
4,Margaret Park,Sales Support Agent,2,2017-05-03,Canada,1584.0
5,Steve Johnson,Sales Support Agent,2,2017-10-17,Canada,1393.92



# Analyzing Sales by Country
* total number of customers
* total value of sales
* average value of sales per customer
* average order value

In [11]:
%%sql
select count(distinct customer_id) AS distinct_customers
from customer
limit 10

Done.


distinct_customers
59


In [12]:
%%sql
SELECT COUNT(DISTINCT c.country) no_countries
from customer c
LEFT JOIN invoice i
USING (customer_id)

Done.


no_countries
24


In [13]:
%%sql
SELECT COUNT(*) no_transactions
from customer c
LEFT JOIN invoice i
USING (customer_id)

Done.


no_transactions
614


In [14]:
%%sql
select c.country, round(sum(total),2) as total_sales_per_country
from customer c
LEFT JOIN invoice i
USING (customer_id)
GROUP BY c.country
ORDER BY round(sum(total),2) DESC


Done.


country,total_sales_per_country
USA,1040.49
Canada,535.59
Brazil,427.68
France,389.07
Germany,334.62
Czech Republic,273.24
United Kingdom,245.52
Portugal,185.13
India,183.15
Ireland,114.84


In [15]:
%%sql
SELECT c.country, COUNT(*) as orders_per_country
from customer c
LEFT JOIN invoice i
USING (customer_id)
GROUP BY 1
ORDER BY 2 DESC

Done.


country,orders_per_country
USA,131
Canada,76
Brazil,61
France,50
Germany,41
Czech Republic,30
Portugal,29
United Kingdom,28
India,21
Chile,13


## How many customers are in each country?
* Where a country has only one customer, collect them into an "Other" group.

In [16]:
%%sql
SELECT c.country, count(distinct c.customer_id) as no_of_customers
from customer c
LEFT JOIN invoice i
USING (customer_id)
group by 1
order by 2 desc

Done.


country,no_of_customers
USA,13
Canada,8
Brazil,5
France,5
Germany,4
United Kingdom,3
Czech Republic,2
India,2
Portugal,2
Argentina,1


### Now group the country column

In [17]:
%%sql
WITH country_customer as(
    SELECT c.country, 
           count(distinct c.customer_id) as no_customers
from customer c
LEFT JOIN invoice i
USING (customer_id)
group by 1
order by 2 desc)

select country, sum(no_customers) AS no_of_customers
from (select case when no_customers <= 1 then 'Other' else 
    country end as country, no_customers
from country_customer) as collection
group by 1
ORDER BY sum(no_customers) DESC
    

Done.


country,no_of_customers
Other,15
USA,13
Canada,8
Brazil,5
France,5
Germany,4
United Kingdom,3
Czech Republic,2
India,2
Portugal,2


In [18]:
%%sql
SELECT
    c.country,
    count(c.customer_id) AS order_per_country, 
    ROUND(SUM(total)) AS total_spend,
    COUNT(DISTINCT c.customer_id) AS no_of_customers
FROM customer c
LEFT JOIN invoice i
USING (customer_id)
GROUP BY 1

Done.


country,order_per_country,total_spend,no_of_customers
Argentina,5,40.0,1
Australia,10,81.0,1
Austria,9,69.0,1
Belgium,7,60.0,1
Brazil,61,428.0,5
Canada,76,536.0,8
Chile,13,97.0,1
Czech Republic,30,273.0,2
Denmark,10,38.0,1
Finland,11,79.0,1


In [19]:
%%sql
WITH condensed AS(SELECT
    c.country,
    count(c.customer_id) AS order_per_country, 
    ROUND(SUM(total)) AS total_spend,
    COUNT(DISTINCT c.customer_id) AS no_of_customers
FROM customer c
LEFT JOIN invoice i
USING (customer_id)
GROUP BY 1)

SELECT
    CASE WHEN no_of_customers <= 1 then 'Other' ELSE country END AS country,
    SUM(order_per_country) AS orders_per_country,
    SUM(total_spend) AS total_spend,
    SUM(no_of_customers) AS no_of_customers
FROM condensed
GROUP BY 1

Done.


country,orders_per_country,total_spend,no_of_customers
Brazil,61,428.0,5
Canada,76,536.0,8
Czech Republic,30,273.0,2
France,50,389.0,5
Germany,41,335.0,4
India,21,183.0,2
Other,147,1093.0,15
Portugal,29,185.0,2
USA,131,1040.0,13
United Kingdom,28,246.0,3


### Force 'Other' to the bottom of your results.

In [20]:
%%sql
WITH condensed AS(SELECT
    c.country,
    count(c.customer_id) AS order_per_country, 
    ROUND(SUM(total),2) AS total_spend,
    COUNT(DISTINCT c.customer_id) AS no_of_customers
FROM customer c
LEFT JOIN invoice i
USING (customer_id)
GROUP BY 1),

final_countries AS(SELECT
    CASE WHEN no_of_customers <= 1 then 'Other' ELSE country END AS country,
    SUM(no_of_customers) AS no_of_customers,
    SUM(order_per_country) AS orders_per_country,
    SUM(total_spend) AS total_spend,
    CASE WHEN no_of_customers <= 1 THEN 1 ELSE 0 END AS sort
FROM condensed
GROUP BY 1)

SELECT country, no_of_customers, orders_per_country, total_spend,
    ROUND(total_spend/no_of_customers,2) AS avg_value_per_customer,
    ROUND(total_spend/orders_per_country,2) AS average_order_value
FROM final_countries
ORDER BY sort ASC, total_spend DESC


Done.


country,no_of_customers,orders_per_country,total_spend,avg_value_per_customer,average_order_value
USA,13,131,1040.49,80.04,7.94
Canada,8,76,535.59,66.95,7.05
Brazil,5,61,427.68,85.54,7.01
France,5,50,389.07,77.81,7.78
Germany,4,41,334.62,83.66,8.16
Czech Republic,2,30,273.24,136.62,9.11
United Kingdom,3,28,245.52,81.84,8.77
Portugal,2,29,185.13,92.56,6.38
India,2,21,183.15,91.58,8.72
Other,15,147,1094.94,73.0,7.45


# Albums vs Individual Tracks
* We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [33]:
%%sql
select count(distinct album_id)
from album

Done.


count(distinct album_id)
347


### How many tracks are in each album?
* there are 347 distinct albums in the database
* 3503 tracks

In [47]:
%%sql
SELECT
    al.album_id,
    count(track_id)no_tracks_per_album,
    min(tr.track_id)first_track_in_album
FROM album al
LEFT JOIN track tr
ON al.album_id = tr.album_id
GROUP BY 1

Done.


album_id,no_tracks_per_album,first_track_in_album
1,10,1
2,1,2
3,3,3
4,8,15
5,15,23
6,13,38
7,12,51
8,14,63
9,8,77
10,14,85


In [31]:
%%sql
select
    i.invoice_id,
    i.customer_id,
    i.invoice_date,
    il.track_id,
    tr.name,
    al.album_id,
    al.title
from invoice i
left join invoice_line il
on i.invoice_id = il.invoice_id
left join track tr
on tr.track_id = il.track_id
left join album al
on al.album_id = tr.album_id
limit 10

Done.


invoice_id,customer_id,invoice_date,track_id,name,album_id,title
1,18,2017-01-03 00:00:00,1158,Right Next Door to Hell,91,Use Your Illusion I
1,18,2017-01-03 00:00:00,1159,Dust N' Bones,91,Use Your Illusion I
1,18,2017-01-03 00:00:00,1160,Live and Let Die,91,Use Your Illusion I
1,18,2017-01-03 00:00:00,1161,Don't Cry (Original),91,Use Your Illusion I
1,18,2017-01-03 00:00:00,1162,Perfect Crime,91,Use Your Illusion I
1,18,2017-01-03 00:00:00,1163,You Ain't the First,91,Use Your Illusion I
1,18,2017-01-03 00:00:00,1164,Bad Obsession,91,Use Your Illusion I
1,18,2017-01-03 00:00:00,1165,Back off Bitch,91,Use Your Illusion I
1,18,2017-01-03 00:00:00,1166,Double Talkin' Jive,91,Use Your Illusion I
1,18,2017-01-03 00:00:00,1167,November Rain,91,Use Your Illusion I


In [37]:
%%sql
with first_tracks_bought as(SELECT
 invoice_id, count(*) no_of_tracks_bought,
    MIN(il.track_id) AS first_track_id
  FROM
    Invoice_Line il
  GROUP BY
    1),
album_tracks as (select al.album_id, count(track_id) tracks_per_album, min(tr.track_id) as first_track
from album al
left join track tr
on al.album_id = tr.album_id
group by 1)


select h.*,case when album_id is null and tracks_per_album is null and first_track is null then 'no' else 'yes' end album_bought
from(select *
from first_tracks_bought fb
left join 
album_tracks at
on fb.first_track_id = at.first_track
and fb.no_of_tracks_bought = at.tracks_per_album) h

Done.


invoice_id,no_of_tracks_bought,first_track_id,album_id,tracks_per_album,first_track,album_bought
1,16,1158,91,16,1158,yes
2,10,201,None,None,None,no
3,2,2516,None,None,None,no
4,8,748,None,None,None,no
5,17,1986,163,17,1986,yes
6,2,30,None,None,None,no
7,11,42,None,None,None,no
8,10,81,None,None,None,no
9,9,196,None,None,None,no
10,2,2663,None,None,None,no


In [51]:
%%sql
select al.album_id,al.title,tr.track_id,tr.name 
from album al
left join track tr
on al.album_id = tr.album_id
limit 20

Done.


album_id,title,track_id,name
1,For Those About To Rock We Salute You,1,For Those About To Rock (We Salute You)
1,For Those About To Rock We Salute You,6,Put The Finger On You
1,For Those About To Rock We Salute You,7,Let's Get It Up
1,For Those About To Rock We Salute You,8,Inject The Venom
1,For Those About To Rock We Salute You,9,Snowballed
1,For Those About To Rock We Salute You,10,Evil Walks
1,For Those About To Rock We Salute You,11,C.O.D.
1,For Those About To Rock We Salute You,12,Breaking The Rules
1,For Those About To Rock We Salute You,13,Night Of The Long Knives
1,For Those About To Rock We Salute You,14,Spellbound


In [45]:
%%sql
with first_tracks_bought as(SELECT
 invoice_id, count(*) no_of_tracks_bought,
    MIN(il.track_id) AS first_track_id
  FROM
    Invoice_Line il
  GROUP BY
    1),
album_tracks as (select al.album_id, count(track_id) tracks_per_album, min(tr.track_id) as first_track
from album al
left join track tr
on al.album_id = tr.album_id
group by 1)

select 
    album_bought,
    count(*) no_invoices,
    round(cast(count(*) as float)/(select count(*)from first_tracks_bought),2) Percentage_of_invoices
from(select h.*,case when tracks_per_album is null and first_track is null then 'no' else 'yes' end album_bought
from(select *
from first_tracks_bought fb
left join 
album_tracks at
on fb.first_track_id = at.first_track
and fb.no_of_tracks_bought = at.tracks_per_album) h)j
group by 1

Done.


album_bought,no_invoices,Percentage_of_invoices
no,498,0.81
yes,116,0.19


* first_tracks_bought: This CTE calculates the first track bought in each invoice and counts the total number of tracks bought in each invoice.
* album_tracks: This CTE calculates the number of tracks per album and identifies the first track for each album.
* The main query joins the results of the two CTEs, first_tracks_bought and album_tracks, based on the condition that the first track bought in each invoice matches the first track of an album, and the number of tracks bought equals the total tracks in the album.
* It then uses a CASE statement to determine whether an album was bought. If the join conditions fail, indicating that either the album ID or the tracks per album or the first track is null, it implies that the album was not bought, and it assigns 'no' to the album_bought column; otherwise, it assigns 'yes'.
* The result set contains all the columns from the joined CTEs along with the album_bought column indicating whether an album was bought or not.

# Analysis of Album vs. Individual Track Purchases

## Findings:
Based on the results of the SQL query:

- Out of the total number of invoices analyzed, 498 invoices (approximately 81%) were categorized as not purchasing albums, while 116 invoices (approximately 19%) were classified as purchasing albums.
- This indicates that a significant majority of the invoices in the Chinook store are comprised of individual track purchases rather than full album purchases.

## Recommendations:
Based on these findings:

- **Optimize Individual Track Sales**: Given that a majority of customers are opting for individual track purchases, the store may focus on optimizing revenue from individual track sales.
- **Promote Popular Tracks**: Focus on promoting popular individual tracks through targeted marketing campaigns, curated playlists, and recommendations.
- **Explore Artist Partnerships**: Explore partnerships with artists to promote singles and popular tracks, potentially leading to increased revenue.
- **Customer Preferences Analysis**: Conduct further analysis to understand customer preferences, listening habits, and trends to tailor the music catalog and promotions accordingly.